In [1]:
#import processing
import ds
import ml
import model_saving
import TS
import TA
# Load the necessary packages and modules
#from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import pandas as pd
import numpy
import numpy as np
import math as m
import talib
import pickle
from sklearn import svm

from hmmlearn.hmm import GaussianHMM
from sklearn.externals import joblib

/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearRegression from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVR from version 0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/octo/anaconda2/envs/carnd-term1/lib/python3.5/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator SVC from version 0.18 when using versi

In [2]:
filename_train ='/home/octo/Dropbox'+ '/SPY9Apr.csv'

In [3]:
# loading csv file
def get_csv_pd(path):
    #spy_pd=pd.read_csv('C:\\Users\Michal\Dropbox\IB_data\SPY.csv',sep=' ',names=['askPrice','askSize','bidPrice','bidSize'],index_col=0,parse_dates=True)
    #spy_pd=pd.read_csv(path+'\SPY.csv',sep=',',names=['askPrice','askSize','bidPrice','bidSize'],index_col=0,parse_dates=True)
    spy_pd=pd.read_csv(path,sep=',',dtype={'askPrice':np.float32,'askSize':np.float32,
                                           'bidPrice':np.float32,'bidSize':np.float32},index_col=0,parse_dates=True)
    #spy_pd = pd.read_csv(path, usecols=['askPrice','askSize','bidPrice','bidSize'], engine='python', skipfooter=3)
    return spy_pd
def BA(df):
    df.bidPrice=df.loc[:,'bidPrice'].replace(to_replace=0, method='ffill')
    df.bidSize=df.loc[:,'bidSize'].replace(to_replace=0, method='ffill')
    df.askPrice=df.loc[:,'askPrice'].replace(to_replace=0, method='ffill')
    df.askSize=df.loc[:,'askSize'].replace(to_replace=0, method='ffill')
    df=df.dropna()
    return df
def preprocessing_mar(df):
    df=df.dropna()
    # to exclude 0
    #data=data[data['bidPrice']>240]
    #data=data[data['askPrice']>240]
    df=df[df['bidPrice']>df.bidPrice.mean()-df.bidPrice.std()]
    df=df[df['askPrice']>df.askPrice.mean()-df.askPrice.std()]
    df['Open']=(df.askPrice+df.bidPrice)/2
    df['Close']=df.Open.shift(1)
    df['High']=df.askPrice.rolling(10).max()
    df['Low']=df.bidPrice.rolling(10).min()
    df['Volume']=df.askSize+df.bidSize
    vwap=((df.loc[:,'bidPrice']*df.loc[:,'bidSize'])+(df.loc[:,'askPrice']*df.loc[:,'askSize']))/(df.loc[:,'bidSize']+df.loc[:,'askSize'])
    df['spread']=(df.Close-vwap)
    #df['Up']=np.where(np.logical_and(df.Close.diff(60)>0.04,df.Close.diff(1)>0.0),1,0)
    #df['Dn']=np.where(np.logical_and(df.Close.diff(60)<-0.04,df.Close.diff(1)<0.0),-1,0)
    #df['UD']=np.where(np.logical_and(df.Close.diff(60)>0.04,df.Close.diff(1)>0.0),1,
    #                 np.where(np.logical_and(df.Close.diff(60)<-0.04,df.Close.diff(1)<0.0),-1,0))
    df['UD']=np.where(df.Close.diff(60)>0.06,1,np.where(df.Close.diff(60)<-0.06,-1,0))
    #df['sc']=np.where(df.Close.diff(60)>0.04,1,np.where(df.Close.diff(60)<-0.04,-1,0))
    df=df.dropna()
    return df

In [4]:
data=get_csv_pd(filename_train)
data=BA(data)
data=preprocessing_mar(data)
data_km=ml.kalman_ma(data)
data['ckm']=data.Close-data_km
data['rsi']=talib.RSI(np.array(data.Close.astype('float64')),timeperiod=10)
data['atr']=talib.ATR(np.array(data.High.astype('float64')),np.array(data.Low.astype('float64')),np.array(data.Close.astype('float64')),timeperiod=10)
data['mom']=talib.MOM(np.array(data.Close.astype('float64')), timeperiod=10)
#data['ret']=np.log(data.Close/data.Close.shift(12))
data=data.dropna()

In [125]:
#len(data)

In [ ]:
#data=data[38000:45300].dropna()

In [127]:
print(data.groupby('UD').size())

UD
-1     5008
 0    79509
 1     3203
dtype: int64


In [ ]:
#print(data.groupby('sc').size())

# ML

### state

In [5]:
from hmmlearn.hmm import GaussianHMM
print("fitting to HMM and decoding ...", end="")
# Make an HMM instance and execute fit
model_hmm = GaussianHMM(n_components=5, covariance_type="full", n_iter=1000)
model_hmm.fit(data[['askPrice', 'askSize', 'bidPrice', 'bidSize','High','Low','spread','ckm','rsi', 'atr','mom']])

fitting to HMM and decoding ...

GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=5, n_iter=1000, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)

In [6]:
from sklearn.externals import joblib
joblib.dump(model_hmm, "markov_hmm.pkl")

['markov_hmm.pkl']

### UD

In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE

In [8]:
X=data[['askPrice', 'askSize', 'bidPrice', 'bidSize','High','Low','spread','ckm','rsi', 'atr','mom']]
#X=data[['askPrice', 'askSize', 'bidPrice', 'bidSize','spread','rsi', 'atr','mom']]
Y=data.UD

In [9]:
sm = SMOTE(random_state=14, ratio ='minority')
X_o,Y_o = sm.fit_sample(X,Y)

In [10]:
SEED=13
rf = RandomForestClassifier(
    n_estimators=11,
    max_features=9,
    random_state=SEED
)

rf.fit(X_o,Y_o)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=1,
            oob_score=False, random_state=13, verbose=0, warm_start=False)

In [147]:
# Kfold validation
import pandas
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

kfold = model_selection.KFold(n_splits=20, random_state=SEED)
results = model_selection.cross_val_score(rf, X_o,Y_o, cv=kfold)
print(results.mean())

0.9271946104728311


In [11]:
filename_rf = 'rf.sav'
pickle.dump(rf, open(filename_rf, 'wb'))